# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [1]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

First, we load up our documents from the `data` directory:

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [4]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [5]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [6]:
chat_history = []
query = "what is azure openai service?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Azure OpenAI is a service provided by Microsoft that offers REST API access to powerful language models developed by OpenAI, including GPT-3, Codex, and Embeddings. These models can be adapted to various tasks such as content generation, summarization, semantic search, and natural language to code translation. The service can be accessed through REST APIs, Python SDK or a web-based interface called Azure OpenAI Studio and offers responsible AI features, virtual network support, content filtering, and managed identity via Azure Active Directory.


We can use this to easy implement chat conversations:

In [7]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = [(query, result["answer"])]
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service provides REST API access to OpenAI's language models such as GPT-3, Codex, and Embeddings. These models can be used for content generation, summarization, semantic search, and natural language to code translation. Azure OpenAI is available through REST APIs, Python SDK, and a web-based interface called Azure OpenAI Studio. In addition, Azure OpenAI comes with virtual network support, managed identity, content filtering, and responsible AI. Access to Azure OpenAI is currently limited to existing Microsoft partners and users who have well-defined use cases.
Question: Which regions does the service support?
Answer: Azure OpenAI Service currently supports the following regions: East US, South Central US, and West Europe.
